# City of L.A. Job Posting Analysis

WARNING: This kernel is incomplete and I likely will not finish it due to other projects having eaten up my time (I got lost in the regexes), but I am providing it as a learning exercise and to share some of my ideas for you to improve your own implementation (See Recommendations, I tried to ensure that the insights I provided there are a unique twist on what has been floating around).

This kernel is created as an entry into the Data Science for Good: City of Los Angeles competition hosted on Kaggle (https://www.kaggle.com/c/data-science-for-good-city-of-los-angeles/overview). 

The city is facing a wave of retirements in 2020, with about 1/3 of the current 50,000 workers eligible for retirement. To ensure the continued effectiveness of public services, the city needs to optimize its job postings for their open positions, such that a more diverse, high-quality applicant pool is obtained, as well as to make promotion structure more transparent and easy to identify.

I will be analyzing the job posting data provided by the City of LA using Python in this Jupyter Notebook, enhanced with the following modules:

* ##### Scikit-learn: Contains open-source machine learning classifiers used to analyze the competition data.
* ##### Pandas: Used to create data tables, modify them, and read/write between data stored in the script and CSV files.
* ##### Numpy: Used to operate on data within matrices more easily and intuitively than in base Python.
* ##### Math: Built-in Python library containing additional mathematical operations such as cosine and flooring.
* ##### Matplotlib: Python library used to create graphs and charts similarly to Matlab.
* ##### Seaborn: Additional plotting library used for creating graphs and other visualizations.
* ##### OS: Built-in Python library used for system operations such as navigating files.
* ##### Re: Python library that allows the use of regular expressions to search for text strings within the job postings.

To make more effective recommendations, I will be using external data on job posting language outcomes to instrument the metrics derived from the competition data. In particular, we will need a way to relate candidate quality and diversity, in addition promotion mobility, to job posting language.

Source of this info: TBD

In [40]:
# Helper libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import os
import re

# Scikit-learn libraries
import sklearn as sk
from sklearn import ensemble
from sklearn import model_selection
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

### Preparing Job Posting Data

As a first step, we want to establish the file paths that will be accessed to retrieve each type of provided data. These paths will start at whichever directory that this script is stored in.

If you wish to add more data on top of the competition's provided data, simply add the appropriate files to the corresponding folders: the program will automatically access each file as needed. 

Also, if you use a different naming scheme for the folders, you can also modify the file paths that are specified in the proceeding line of code:

In [2]:
job_post_path = "/CityofLA/Job Bulletins/" # Stores .txt files

The following function will be reused in order to retrieve the paths of the files stored in a given directory. It takes two arguments, a string of the path to the stored directory, and a string of the file extension to check for (only files of the specified type will be scanned). These files will be retrieved in alphanumerical order.

In [82]:
def getFiles(fileDirectory, fileType):
    fileNames = []
    filePaths = []
    files = os.listdir(fileDirectory)
    print("Number of job postings found: ", len(files))
    files.sort()
    for file in files:
        if file.endswith(fileType):
            fileNames.append(file)
            filePaths.append(fileDirectory + file)
    return (fileNames, filePaths)

Finally, before proceeding, we will need to get the system path leading to where this kernel is stored. The aforementioned folder paths must be stuck onto the end of this base path in order for the script to work.

In [83]:
system_path = os.path.dirname(os.path.realpath("Job Posting Analysis"))
print(system_path)

/home/jovyan/work/Documents/github_repos/LA Job Text Analysis


#### Testing File Paths

The following block of code should display the heading of the first job posting file stored within the desired folder; ensure that this is the case on your file system before proceeding!

In [84]:
jobFileNames, jobFilePaths = getFiles(system_path + job_post_path,'.txt')
first_job_post = jobFilePaths[0]
print("File path of first job posting: ", first_job_post)
file = open(first_job_post,'r')
first_contents = file.readlines()
print("Header: ", first_contents[0])

Number of job postings found:  684
File path of first job posting:  /home/jovyan/work/Documents/github_repos/LA Job Text Analysis/CityofLA/Job Bulletins/311 DIRECTOR  9206 041814.txt
Header:  311 DIRECTOR



### Setting Up File Reading and Submission

Here I will define functions for parsing data from each individual text file, as well as setting up the submission CSV file based on the sample job class and titles files provided. As a precaution, we want to first check that the proper number of job titles were provided in the job_titles.csv file:

In [85]:
sample_job_title_path = system_path + "/CityofLA/Additional data/job_titles.csv"
titles = pd.read_csv(sample_job_title_path)
title_list = []
title_list.append("311 DIRECTOR")
for title in titles["311 DIRECTOR"]:
    title_list.append(title)
print("Provided job list has %d job titles, while %d job postings were found in the folder." 
      % (len(title_list),len(jobFilePaths)))

Provided job list has 668 job titles, while 683 job postings were found in the folder.


Since there is a discrepancy in the numbers of jobs listed in the CSV file and actual postings, I will ignore the job_titles.csv data and instead retrieve job titles from each file.

First, I will be setting up the submission CSV file based on the file structure provided in the Kaggle competition. If you wish to use an already created CSV file, simply change the variable 'Setup' to False.

In [86]:
Setup = True
if Setup == True:
    sample_job_class_path = system_path + "/CityofLA/Additional data/sample job class export template.csv"
    
    classes = pd.read_csv(sample_job_class_path)

    submission = pd.DataFrame(columns=classes.columns)
else:
    submission = pd.read_csv("kaggle_submission.csv")
submission.head()

,FILE_NAME,JOB_CLASS_TITLE,JOB_CLASS_NO,REQUIREMENT_SET_ID,REQUIREMENT_SUBSET_ID,JOB_DUTIES,EDUCATION_YEARS,SCHOOL_TYPE,EDUCATION_MAJOR,EXPERIENCE_LENGTH,...,COURSE_LENGTH,COURSE_SUBJECT,MISC_COURSE_DETAILS,DRIVERS_LICENSE_REQ,DRIV_LIC_TYPE,ADDTL_LIC,EXAM_TYPE,ENTRY_SALARY_GEN,ENTRY_SALARY_DWP,OPEN_DATE


#### Extracting Features

With the submission set up as intended, we are now ready to begin text analysis to extract the desired features as stipulated in the "kaggle_data_dictionary.csv" file. These headers are already incorporated into the work-in-progress submission dataframe, as created and shown above.

As a first step, we want to retrieve the texts of each job posting so that we can apply regex searches to extract features.

NOTE: On my Mac, the text files were encoded with ISO-8859-1 instead of UTF-8, which led to problems with retrieving é characters, I set an alternative encoding accordingly.

In [90]:
jobTexts = []
enc = "ISO-8859-1"

for i in range(len(jobFilePaths)):
    file = open(jobFilePaths[i], encoding=enc)
    jobText = file.read()
    # Strip newlines and tabs and replace them with spaces
    #jobText = jobText.replace('\n',' ')
    #jobText = jobText.replace('\t',' ')
    jobTexts.append(jobText)
print(jobTexts[0])

311 DIRECTOR
Class Code:       9206
Open Date:  04-18-14
ANNUAL SALARY 

$125,175 to $155,514 

DUTIES

A 311 Director is responsible for the successful operation and expansion of the 311 Call Center in the Information Technology Agency (ITA) which answers call from constituents regarding Citywide services provided by City departments; works to ensure the efficient and effective resolution of any issues that may arise; plans, directs, hires, coaches, and coordinates a large staff of professional, technical and clerical employees engaged in the implementation, administration, and operations of the City's 311 Call Center; applies sound supervisor principles and techniques in building and maintaining and effective work force; fulfills equal opportunity responsibilities; and does related work.

REQUIREMENTS

1. One year of full-time paid experience as a Senior Management Analyst with the City of Los Angeles or in a class which is at least at the level which provides professional experience

Now, we will run through a series of scripts to populate one to several submission fields each from the text data. Some of these will be more involved than others, and thus more prone to error, and so I wish to keep these self-contained as much as possible. There are 25 fields that must be filled in total, with the JOB_CLASS_TITLE already retrieved.

In [126]:
def findTitle(text):
    regex = re.compile(r'.*')
    title = re.search(regex, text)
    if title:
        return title.group(0)
    else:
        return None

def findCode(text):
    regex = re.compile(r'(?:Class\s+[C,c]ode:)(?:\s+)(\d\d\d\d)')
    code = re.search(regex, text)
    if code:
        return code.group(1)
    else:
        return None

def findDate(text):
    regex = re.compile(r'(?:Open\s+[D,d]ate:)?(?:\s+)?(\d*-\d*-\d*)')
    date = re.search(regex, text)
    if date:
        return date.group(1)
    else:
        return None
    
def findSalary(text):
    regex = re.compile(r'\$(\d+,\d+)((\s(to|and)\s)(\$\d+,\d+))?')
    salary = re.search(regex, text)
    if salaray:
        return salary.group(1)
    else:
        return None

# The requirement sets are denoted with a numbered list if there are multiple, or none
# if there is only 1 requirement set. Subsets are denoted with a lettered list under each
# number, if any. Subset field should default to 'A' if there is no subset specified.
def findReqs(text):
    # regex1 finds the entire requirements section
    regex1 = re.compile(r'(?:Open [D,d]ate:)(\s+)(\d\d-\d\d-\d\d)')
    reqSection = re.search(regex1, text)
    # regex2 splits the requirements section into individual listed points
    regex2 = re.compile(r'')
    reqSets = [['A']]
    return (reqSets, reqSubsets,reqTexts)

In [127]:
classCodes = []
openDates = []
fileNames = []
jobTitles = []
for i in range(len(jobTexts)):
    jobText = jobTexts[i]
    jobFile = jobFileNames[i]
    #print(jobFile)
    jobTitle = findTitle(jobText)
    classCode = findCode(jobText)
    openDate = findDate(jobText)
    salary = findSalary(jobText)
    
    fileNames.append(jobFile)
    jobTitles.append(jobTitle)
    classCodes.append(classCode)
    openDates.append(openDate)
submission["JOB_CLASS_TITLE"] = jobTitles
submission["JOB_CLASS_NO"] = classCodes
submission["OPEN_DATE"] = openDates
submission["FILE_NAME"] = fileNames
submission.head(5)

,FILE_NAME,JOB_CLASS_TITLE,JOB_CLASS_NO,REQUIREMENT_SET_ID,REQUIREMENT_SUBSET_ID,JOB_DUTIES,EDUCATION_YEARS,SCHOOL_TYPE,EDUCATION_MAJOR,EXPERIENCE_LENGTH,...,COURSE_LENGTH,COURSE_SUBJECT,MISC_COURSE_DETAILS,DRIVERS_LICENSE_REQ,DRIV_LIC_TYPE,ADDTL_LIC,EXAM_TYPE,ENTRY_SALARY_GEN,ENTRY_SALARY_DWP,OPEN_DATE
0,311 DIRECTOR 9206 041814.txt,311 DIRECTOR,9206,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04-18-14
1,ACCOUNTANT 1513 062218.txt,ACCOUNTANT,1513,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,06-22-18
2,ACCOUNTING CLERK 1223 071318.txt,ACCOUNTING CLERK,1223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,07-13-18
3,ACCOUNTING RECORDS SUPERVISOR 1119 072718.txt,ACCOUNTING RECORDS SUPERVISOR,1119,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,07-27-18
4,ADMINISTRATIVE ANALYST 1590 060118.txt,ADMINISTRATIVE ANALYST,1590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,06-01-18


## Promotion Alerts

It is desired that the system administrator should be able to search up a specific employee and, based upon their job and length of employment, return any promotions that they are currently eligible for, along with any additional requirements these promotions have.

The helper function to do this, given the job postings dataframe, a CSV-derived Pandas employee dataframe, and the employee name string to search for (this can be adapted to employee id or whichever unique identifier your system uses), is included below.

In [ ]:
def findPromotions(job_postings, data, employee):
    promotionNotices = []
    employee_line = data.loc[employee,:]
    years = float(employee_line[years])
    job = employee_line[job]
    possiblePromotions = job_postings[(EXP_JOB_CLASS_TITLE == job) & 
                                      (EXPERIENCE_LENGTH <= years)]
    promotions = possiblePromotions[JOB_CLASS_TITLE]
    # Put all types of desired requirements here:
    requirements = []
    for i in range(len(promotions)):
        req = ""
        requirements.append(req)
    for i in range(len(promotions)):
        notice = "Eligible for promotion to %s: also requires %s" % promotions[i],requirements[i]
        promotionNotices.append(notice)
    return promotionNotices

I will create a test file with a few employees to verify that the function works as intended:

In [ ]:
employee_test = pd.DataFrame()
# TODO

## Recommendations

### To Improve Candidate Diversity and Quality
1. First, make sure to scrub the job descriptions of unexplained acronyms or jargon; this would intuitively bias the candidate pool towards those already intimately familiar with the specific job functions and protocols at a given organization and discourages diversity. In code, check for any instances consecutive capital letters (excluding headers), for example DMV, that may be acronyms and examine if they are properly explained. You might also hard code checks for jargon indicators such as 'Department' or 'Section #'.
2. Ensure that readability scores are low enough to be understandable by most of the target audience, particularly for low-requirement jobs. (Find a way to graph readability vs. experience level?)
3. On top of a simple search for gendered nouns/pronouns, check for gendered items in job description.

### Promotion Paths
1. As an improvement on the diagrams provided, create a function to autofill a promotion graph for a desired job to make a clear visualisation